In [1]:
import psycopg2 as pg
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option("display.max_rows", 100)

In [2]:
connection = pg.connect("host=localhost dbname=autos_db user=postgres")

In [3]:
df = pd.read_sql('SELECT * FROM carfirst_auction WHERE highest_bid > 0', con=connection)
df["vehicle_name"] = df["title"] + " " + df["subtitle"]

/home/sher/Experiments/sher_autos/api/venv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [4]:
df["highest_bid"] = df["highest_bid"]/100000
df.head()

,id,auction_id,start_time,end_time,title,subtitle,start_price,bids_placed,bidders,location,...,make,model,transmission_type,fuel_type,registration_year,reference_id,created_at,updated_at,highest_bid,vehicle_name
0,4,199c4022-7b36-4131-8813-57cb697782fb,2022-03-24 07:00:00+00:00,2022-03-24 07:10:00+00:00,Honda Civic 2010,1.8 i-VTEC Oriel Prosmatec,1674000,37,8,Islamabad,...,Honda,Civic,Automatic,Petrol,2010,PK-1177619,2022-03-24 08:36:39.135029+00:00,2022-03-24 09:32:42.698099+00:00,18.94,Honda Civic 2010 1.8 i-VTEC Oriel Prosmatec
1,12,ee2f05d1-6da6-4249-90b4-ee69f00ab726,2022-03-24 07:00:00+00:00,2022-03-24 07:10:00+00:00,Toyota Vitz 2011,1.0 F,1397000,58,12,Lahore,...,Toyota,Vitz,Automatic,Petrol,2015,PK-1177611,2022-03-24 08:36:39.135622+00:00,2022-03-24 09:32:50.062380+00:00,16.87,Toyota Vitz 2011 1.0 F
2,18,7f6f50fe-22be-4d10-ae8c-8a03dd3c1d85,2022-03-24 07:45:00+00:00,2022-03-24 07:55:00+00:00,Honda City 2021,1.5 i-VTEC Aspire Prosmatec,2711000,90,13,Karachi,...,Honda,City,Automatic,Petrol,2022,PK-1177616,2022-03-24 08:36:39.160490+00:00,2022-03-24 09:32:51.257614+00:00,34.14,Honda City 2021 1.5 i-VTEC Aspire Prosmatec
3,33,8db71229-1477-4383-b2a3-196e2e4a9dd5,2022-03-24 07:15:00+00:00,2022-03-24 07:25:00+00:00,Honda City 2017,1.5 i-VTEC Aspire Prosmatec,2342000,30,13,Islamabad,...,Honda,City,Automatic,Petrol,2017,PK-1177613,2022-03-24 08:36:44.042753+00:00,2022-03-24 09:33:02.168376+00:00,25.70,Honda City 2017 1.5 i-VTEC Aspire Prosmatec
4,17,5a3c53aa-4c51-4cc9-b1a5-065ee4a2d89d,2022-03-24 07:45:00+00:00,2022-03-24 07:55:00+00:00,Toyota Corolla 2011,1.3 GLI,1185000,37,7,Islamabad,...,Toyota,Corolla,Manual,Petrol,2011,PK-1177623,2022-03-24 08:36:39.160394+00:00,2022-03-24 09:32:51.240667+00:00,13.90,Toyota Corolla 2011 1.3 GLI


In [5]:
print(f'Total Auctions: {len(df)}')

Total Auctions: 215


# Competitive Auctions

In [6]:
df.sort_values(by="bids_placed", ascending=False)[["title", "subtitle", "bids_placed", "bidders", "location", "transmission_type", "registration_year", "color", "highest_bid"]].head()

,title,subtitle,bids_placed,bidders,location,transmission_type,registration_year,color,highest_bid
2,Honda City 2021,1.5 i-VTEC Aspire Prosmatec,90,13,Karachi,Automatic,2022,White,34.14
203,Honda N 2016,660cc WGN Custom,76,15,Lahore,Automatic,2018,White,17.61
112,Suzuki Liana 2006,1.6,64,4,Karachi,Manual,2006,Grey,7.12
182,Suzuki Swift 2017,1.3 DLX with Navigation,62,7,Karachi,Automatic,2017,Silver,17.36
7,Suzuki Swift 2012,1.3 DLX,62,14,Lahore,Manual,2012,Grey,11.76


# Popular Auctions

In [7]:
df.sort_values(by="bidders", ascending=False)[["title", "subtitle", "highest_bid", "bids_placed", "bidders", "location", "transmission_type", "registration_year", "color",]].head()

,title,subtitle,highest_bid,bids_placed,bidders,location,transmission_type,registration_year,color
135,Toyota Corolla 2017,1.6 Altis Facelift Super ECT,32.84,52,17,Islamabad,Automatic,2017,White
203,Honda N 2016,660cc WGN Custom,17.61,76,15,Lahore,Automatic,2018,White
7,Suzuki Swift 2012,1.3 DLX,11.76,62,14,Lahore,Manual,2012,Grey
51,Daihatsu Mira 2016,660cc X,15.55,54,14,Islamabad,Manual,2021,Red
167,Suzuki Wagon R 2018,1.0 VXR,15.35,41,13,Lahore,Manual,2018,Silver


# Honda Summary

In [8]:
df_toyota = df[df["make"] == "Honda"]
df_toyota.groupby(["model", "subtitle", "registration_year", "transmission_type",]).agg({'highest_bid': ['min', 'max', 'mean', 'median', 'count']})

highest_bid  \
                                                                                min   
model subtitle                      registration_year transmission_type               
Acty  660cc                         2016              Automatic             7.84000   
City  1.3 EXI S                     2000              Manual                4.14000   
      1.3 I-VTEC                    2019              Manual               24.02000   
      1.3 i-DSI                     2005              Manual               10.10000   
      1.3 i-VTEC                    2013              Manual               17.36000   
                                    2014              Manual               16.64000   
                                    2015              Manual               18.00005   
                                    2016              Manual               17.95000   
                                    2018              Manual               18.92000   
      1.3 i-VTEC Aspire             2016              Manual               17.65000   
      1.3 i-VTEC Aspire Prosmatec   2016              Automatic            24.02000   
      1.5 V-TECH Steermatic         2007              Automatic            11.28000   
      1.5 i-VTEC Aspire Prosmatec   2013              Automatic            17.21000   
                                    2017              Automatic            25.70000   
                                    2022              Automatic            34.14000   
Civic 1.5 EX - EG                   1995              Manual                3.48000   
      1.5 EX-EG                     1994              Manual                4.46000   
      1.5 EXI                       2007              Manual                7.61000   
      1.5 EXi - New Shape           2005              Manual                8.14000   
      1.5 EXi Prosmatec             2005              Automatic            11.03000   
      1.5 i-VTEC Turbo              2016              Automatic            32.30000   
      1.6 VTi Oriel                 2006              Manual               11.71000   
      1.6 VTi- EK                   2000              Manual                6.12000   
      1.8 I-VTEC Oriel CVT UG       2021              Automatic            44.67000   
      1.8 i-VTEC Oriel CVT UG       2017              Automatic            29.40000   
                                    2018              Automatic            34.25000   
                                    2019              Automatic            34.38000   
                                    2021              Automatic            41.48000   
      1.8 i-VTEC Oriel Prosmatec    2008              Automatic            15.38000   
                                    2010              Automatic            18.94000   
                                    2011              Automatic            15.00000   
      1.8 i-VTEC Oriel Prosmatec UG 2013              Automatic            20.05000   
      1.8 i-VTEC Oriel-Rebirth      2012              Manual               17.51000   
      1.8 i-VTEC Prosmatec          2011              Automatic            14.89000   
Fit   1.5 F                         2019              Automatic            24.81000   
N     660cc WGN                     2016              Automatic            13.62000   
      660cc WGN Custom              2018              Automatic            17.61000   
      660cc WGN Custom Turbo        2017              Automatic            15.74000   
Vezel 1.5 Hybrid X                  2020              Automatic            30.88000   
      1.5 Hybrid Z                  2015              Automatic            25.19000   

                                                                                   \
                                                                              max   
model subtitle                      registration_year transmission_type             
Acty  660cc                         2016              Automatic           7

# Toyota Summary

In [9]:
df_toyota = df[df["make"] == "Toyota"]
df_toyota.groupby(["model", "subtitle", "registration_year", "transmission_type",]).agg({'highest_bid': ['min', 'max', 'mean', 'median', 'count']})

highest_bid  \
                                                                                               min   
model        subtitle                              registration_year transmission_type               
Aqua         1.5 G Soft Leather                    2018              Automatic               28.60   
             1.5 L                                 2016              Automatic               19.96   
                                                   2017              Automatic               16.72   
             1.5 S Push Start                      2016              Automatic               21.73   
Axio         1.5 G                                 2012              Automatic               20.95   
             1.5 Hybrid G WxB                      2021              Automatic               43.90   
Belta        1.0 X                                 2008              Automatic               10.20   
CHR          1.8 Hybrid S                          2018              Automatic               42.64   
Corolla      1.3 GLI                               2005              Manual                   9.12   
                                                   2010              Manual                  14.95   
                                                   2011              Manual                  13.90   
                                                   2012              Manual                  16.38   
                                                   2013              Manual                  14.83   
                                                   2014              Manual                  14.40   
                                                   2020              Manual                  22.31   
             1.3 GLI - Limited Edition - Old Shape 2015              Manual                  20.11   
             1.3 GLI -Super ECT                    2015              Automatic               21.28   
             1.3 GLI Facelift                      2018              Manual                  26.40   
             1.3 GLI New Key                       2016              Manual                  23.94   
             1.3 GLI New Key Super ECT             2017              Automatic               25.37   
             1.3 GLI New Shape                     2014              Manual                  21.33   
             1.3 GLI Super ECT                     2017              Automatic               23.88   
                                                   2018              Automatic               27.53   
                                                   2019              Automatic               24.17   
             1.3 XLI                               2006              Manual                  10.67   
                                                   2009              Manual                  11.09   
                                                   2011              Manual                  13.94   
                                                   2016              Manual                  20.57   
                                                   2019              Manual                  24.44   
             1.3 XLI - New Shape                   2014              Manual                  19.33   
             1.3 XLI - Old Shape                   2014              Manual                  15.02   
             1.3 XLI- Old Shape                    2008              Manual                   8.99   
             1.6 Altis Facelift Super ECT          2017              Automatic               29.73   
             1.6 Altis SR Cruisetronic             2013              Automatic               23.98   
             1.6 Altis Super ECT                   2018              Automatic               29.37   
             1.6 GLI Super ECT                     2012              Automatic               18.78   
             1.6 SE Saloon                         2005              Manual                   8.36   
       

# Suzuki Summary

In [10]:
df_toyota = df[df["make"] == "Suzuki"]
df_toyota.groupby(["model", "subtitle", "registration_year", "transmission_type",]).agg({'highest_bid': ['min', 'max', 'mean', 'median', 'count']})

highest_bid  \
                                                                            min   
model   subtitle                registration_year transmission_type               
Alto    1.0 VXR                 2000              Manual                   3.16   
                                2004              Manual                   2.91   
                                2005              Manual                   4.51   
                                2007              Manual                   4.34   
                                2008              Manual                   5.25   
                                2010              Manual                   5.23   
                                2011              Manual                   5.70   
                                2012              Manual                   5.93   
        660cc Eco S             2017              Automatic               14.17   
        660cc VXL AGS           2019              Automatic               17.07   
                                2020              Automatic               15.21   
                                2021              Automatic               19.26   
        660cc VXR               2020              Manual                  15.07   
                                2021              Manual                  16.04   
Bolan   800cc VX                2017              Manual                   6.53   
Cultus  1.0 Limited Edition     2016              Manual                  11.18   
        1.0 VXL                 2020              Manual                  19.14   
        1.0 VXL AGS             2019              Automatic               19.88   
                                2020              Automatic               21.08   
        1.0 VXLi                2010              Manual                   7.70   
        1.0 VXR                 2004              Manual                   3.12   
                                2018              Manual                  16.35   
        1.0 VXR Euro II         2013              Manual                   8.49   
                                2014              Manual                   9.62   
                                2015              Manual                  10.98   
                                2016              Manual                   8.94   
        1.0 VXRI                2008              Manual                   6.93   
        1.0 VXRi                2007              Manual                   4.86   
                                2010              Manual                   5.82   
                                2011              Manual                   7.08   
Every   660cc 2 Pwr             2015              Automatic                7.69   
                                2017              Automatic               12.37   
Liana   1.3 LXI                 2007              Manual                   6.62   
        1.3 RXI                 2006              Manual                   5.14   
        1.6                     2006              Manual                   7.12   
Mehran  800cc VX                2007              Manual                   4.52   
                                2010              Manual                   3.46   
                                2012              Manual                   5.56   
        800cc VX Euro II        2015              Manual                   5.31   
                                2016              Manual                   5.50   
Spacia  660cc X                 2021              Automatic               11.46   
Swift   1.3 DLX                 2012              Manual                  11.39   
                                2013              Manual                  13.66   
                                2015              Manual                  13.86   
                                2017              Manual                  13.00   
        1.3 DLX with Navigation 2017              Automatic               17.36   
W